In [3]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [4]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import datetime as dt
from sklearn.cluster import KMeans
import random as ran
from sklearn.metrics.pairwise import euclidean_distances as euc
from functools import reduce

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

from src import crs, PublicPredictor, PrevPublicPredictor
from IPython.display import clear_output, Markdown, display

In [5]:
# 1. load_excel
data_path = "data/apt_all.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

In [6]:
# 2. data preprocessing
p, m = crs.utils.data_preprocessing(xlsx)

m.set_index("month", inplace=True)
m.head()

/Users/formegusto/Desktop/idoformegusto/development/research/what-is-public-fee/src/crs/utils/data_preprocessing.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  datas_df[household_name] = household_df[col][3:].to_list()
/Users/formegusto/Desktop/idoformegusto/development/research/what-is-public-fee/src/crs/utils/data_preprocessing.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  month_usage_df[name] = [


,아파트1-101-1002,아파트1-101-104,아파트1-101-1102,아파트1-101-1202,아파트1-101-1302,아파트1-101-1401,아파트1-101-1403,아파트1-101-1502,아파트1-101-1504,아파트1-101-1602,...,아파트4-104-101,아파트4-104-1301,아파트4-104-1501,아파트4-104-301,아파트4-105-102,아파트4-105-103,아파트4-105-1102,아파트4-105-1104,아파트4-105-601,아파트4-105-803
month,,,,,,,,,,,,,,,,,,,,,
1,253,689,183,239,455,343,380,219,389,350,...,335,363,181,216,270,518,230,279,161,156
2,251,481,177,218,397,297,339,197,348,288,...,309,339,146,184,267,344,208,236,167,150
3,255,424,203,222,378,303,349,202,357,368,...,313,372,152,236,268,417,212,272,166,155
4,252,355,180,230,368,278,298,188,346,365,...,266,346,149,210,268,421,208,243,156,148
5,230,357,258,203,414,270,418,156,320,265,...,305,256,151,222,287,349,185,284,165,180


# 최신 버전
- 전기요금계 가격이 1000원 미만일 경우 1000원으로 최저요금으로 맞추어 버리는 경우가 있음.
- 그래서 해당 가구들은 수거 과정에서 제외 시켜야 함
    - 기본요금, 전력량요금 : 수거 과정에서 제외
    - 필수사용량보장공제 : 제외
    - 기후환경요금, 연료비조정액 : 수거 과정에서 제외했으므로, 해당 단계도 세대부 사용량에서 제외하고 percentage 진행

In [240]:
predict_test = pd.DataFrame(columns=['실제 요금', '예측 요금', '오차'])
households_columns = list()

for _ in range(0, 500):
    n = ran.randrange(50, len(m.columns) + 1)
    _m = m.sample(n=n, axis=1)
    _month = ran.randrange(1, 13)

    month_df = pd.DataFrame(_m.loc[_month])
    month_df.reset_index(inplace=True)
    month_df.columns = ['name', 'usage (kWh)']
    
    PUBLIC_PERCENTAGE = ran.randrange(10, 60)
    households_kWh = month_df['usage (kWh)'].sum()
    APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))

    pp = PublicPredictor(
        APT=APT,
        month_df=month_df,
        month=_month
    )

    predict_test = predict_test.append(pp.predict, ignore_index=True)
    households_columns.append((PUBLIC_PERCENTAGE,_month,month_df['name'].values))

In [241]:
max_err = predict_test['오차'].abs().max()
min_err = predict_test['오차'].abs().min()
predict_rate = predict_test.apply(lambda x: 
   x['예측 요금'] / x['실제 요금'] * 100 if x['실제 요금'] > x['예측 요금'] 
   else x['실제 요금'] / x['예측 요금'] * 100
   , axis=1).mean()
display(Markdown("# 공용부 요금 예측 시뮬레이션 결과 입니다.\n\n"))

print("• 최고 오차 : {}원".format(format(max_err,",")))
print("• 최소 오차 : {}원".format(format(min_err,",")))
print("• 평균 적중률 : {}%".format(format(predict_rate,",")))

predict_test.head()

# 공용부 요금 예측 시뮬레이션 결과 입니다.



• 최고 오차 : 4,400원
• 최소 오차 : 410원
• 평균 적중률 : 99.9294566444658%


,실제 요금,예측 요금,오차
0,10711500,10708240,3260
1,2435070,2433270,1800
2,9963600,9960360,3240
3,5214890,5212450,2440
4,3624730,3622790,1940


# 예전 버전
- 최고오차가 엄청 크게 나오는 현상 발생

In [7]:
prev_predict_test = pd.DataFrame(columns=['실제 요금', '예측 요금', '오차'])
households_columns = list()

for _ in range(0, 500):
    n = ran.randrange(50, len(m.columns) + 1)
    _m = m.sample(n=n, axis=1)
    _month = ran.randrange(1, 13)

    month_df = pd.DataFrame(_m.loc[_month])
    month_df.reset_index(inplace=True)
    month_df.columns = ['name', 'usage (kWh)']
    
    PUBLIC_PERCENTAGE = ran.randrange(10, 60)
    households_kWh = month_df['usage (kWh)'].sum()
    APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))

    pp = PrevPublicPredictor(
        APT=APT,
        month_df=month_df,
        month=_month
    )

    prev_predict_test = prev_predict_test.append(pp.predict, ignore_index=True)
    households_columns.append((PUBLIC_PERCENTAGE,_month,month_df['name'].values))

In [8]:
max_err = prev_predict_test['오차'].abs().max()
min_err = prev_predict_test['오차'].abs().min()
predict_rate = prev_predict_test.apply(lambda x: 
   x['예측 요금'] / x['실제 요금'] * 100 if x['실제 요금'] > x['예측 요금'] 
   else x['실제 요금'] / x['예측 요금'] * 100
   , axis=1).mean()

display(Markdown("# 공용부 요금 예측 시뮬레이션 결과 입니다.\n\n"))

print("• 최고 오차 : {}원".format(format(max_err,",")))
print("• 최소 오차 : {}원".format(format(min_err,",")))
print("• 평균 적중률 : {}%".format(format(predict_rate,",")))

prev_predict_test.head()

# 공용부 요금 예측 시뮬레이션 결과 입니다.



• 최고 오차 : 232,950원
• 최소 오차 : 10원
• 평균 적중률 : 99.28338265772311%


,실제 요금,예측 요금,오차
0,2531560,2534040,-2480
1,1800120,1801790,-1670
2,2843690,2843810,-120
3,3746580,3751230,-4650
4,1726290,1723490,2800


In [15]:
prev_predict_test[prev_predict_test['오차'].abs() > 232720]

,실제 요금,예측 요금,오차
460,1391330,1624280,-232950


In [17]:
PUBLIC_PERCENTAGE, _month, h_cols = households_columns[460]

_m = m[h_cols]

month_df = pd.DataFrame(_m.loc[_month])
month_df.reset_index(inplace=True)
month_df.columns = ['name', 'usage (kWh)']

households_kWh = month_df['usage (kWh)'].sum()
APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))

pp = PrevPublicPredictor(
    APT=APT,
    month_df=month_df,
    month=_month
)

pp.predict

실제 요금    1391330
예측 요금    1624280
오차       -232950
dtype: int64

In [22]:
import json

In [24]:
err_data_save = dict({
    "공용부 percentage": PUBLIC_PERCENTAGE,
    "월": _month,
    "가구구성": h_cols.tolist()
})

with open("test_data.json", 'w') as f:
    json.dump(err_data_save, f)

In [25]:
pp.basic

46810

In [26]:
pp.elec_rate

847734